In [147]:
from bs4 import BeautifulSoup as soup 
from urllib.request import urlopen as uReq
from urllib.request import Request
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import numpy as np

In [148]:
myurl =  'https://cedh-decklist-database.com'
cards = [] # initializes list of all cards

In [149]:
client = uReq(myurl) # opening connection 
page_html = client.read() # content to variable
client.close() # closes connection 
page_soup = soup(page_html, 'html.parser') # html parsing 
containers = page_soup.findAll("ul",{"class": "ddb-decklists"})
htmls = []
for c in containers:
    htmls.append(c.a["href"])
htmls

['https://www.moxfield.com/decks/Bv8Jo8vkCUmHo-If-g6hug',
 'https://www.moxfield.com/decks/boQZOdVCC02uAjGpjShtLA',
 'https://www.moxfield.com/decks/C5xyxwytW0eaKZkCPtkbrg',
 'https://www.moxfield.com/decks/PFGBjM5BnESfH9F8qBIIaw',
 'https://www.moxfield.com/decks/Vva2blXPDEWTPCIW7QfpbA',
 'https://jhoira.moxfield.com/',
 'https://www.moxfield.com/decks/CwZ4jijMkkWpvx7smX2-jQ',
 'https://www.moxfield.com/decks/fsrJzDCtX0-8UZczYGcwDQ/',
 'https://www.moxfield.com/decks/Ns1U8bi1fEap3ZT7Is5WmA',
 'https://www.moxfield.com/decks/ksOWqeElwUaNmAeybXoEjw',
 'https://www.moxfield.com/decks/DfN8K5u8CUCuaGp_l9a2PQ',
 'https://tappedout.net/mtg-decks/tyrant-queen/',
 'https://www.moxfield.com/decks/ozNqMp_H90mIuyhnKLXI4g',
 'https://www.moxfield.com/decks/7z2H2haMwUaDAyB2pjWrUA',
 'https://www.moxfield.com/decks/xmH0i1HMHUi2iCd279RYPA',
 'https://www.moxfield.com/decks/rVEjo7IdpECuP96MlsjcrQ',
 'https://www.moxfield.com/decks/W3Ko6od8a0CJ5T2Kd1ftdQ',
 'https://www.moxfield.com/decks/fg8M7HBUDUub1

In [150]:
for decklist in htmls:
    if 'moxfield' in decklist: # checks if its a moxfield deck
        chromedriver_path= r"C:\Users\Brian\OneDrive\Desktop\chromedriver.exe"
        driver = webdriver.Chrome(chromedriver_path)
        driver.get(decklist)
        time.sleep(3) #if you want to wait 3 seconds for the page to load
        page_source = driver.page_source
        driver.close()
        page_soup = soup(page_source, 'lxml')
        containers = page_soup.findAll("tr",{"class": "table-deck-row"})
        for c in containers:
            cards.append( c.a.get_text())
        cards = cards[:len(cards)-8]
    if 'tappedout' in decklist:#checks if its a tapped out deck
        chromedriver_path= r"C:\Users\Brian\OneDrive\Desktop\chromedriver.exe"
        driver = webdriver.Chrome(chromedriver_path)
        driver.get(decklist)
        time.sleep(3) #if you want to wait 3 seconds for the page to load
        page_source = driver.page_source
        driver.close()
        page_soup = soup(page_source, 'lxml')
        containers = page_soup.findAll("li",{"class": "member"})
        for c in containers:
            if '\n' in c.span.a.get_text():
                card = c.span.a.get_text()[16:].replace('\n','').strip()
                cards.append(card)

In [151]:
card_data = pd.DataFrame({'Card Name': cards}) # creates a data frame of all cards
card_data.to_csv('cards3s.csv') # exports as csv

In [152]:
card_data = pd.DataFrame(card_data.value_counts()) # counts number of each card and makes new dataframe
card_data.rename( columns={0 :'Occurances'}, inplace=True ) # sets coorrect coolumn names
card_data = card_data[card_data['Occurances'] > 10] # removes fringe cards
card_data.to_csv("CEDH_Staple_Ranking1.csv") # output of ranks

In [154]:
card_data.head(10)

,Occurances
Card Name,
Chrome Mox,217
Mana Crypt,216
Sol Ring,214
Mox Diamond,210
Polluted Delta,185
Command Tower,184
Flooded Strand,178
Misty Rainforest,174
Scalding Tarn,174
